In [2]:
!pip install openai

In [3]:
!pip install tqdm

In [4]:
import openai
import os
import pandas as pd
import time
from tqdm import tqdm

#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = "API-KEY" # provide your own api here

ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/charset_normalizer/md.cpython-311-darwin.so, 2): no suitable image found.  Did find:
	/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/charset_normalizer/md.cpython-311-darwin.so: mach-o, but wrong architecture
	/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/charset_normalizer/md.cpython-311-darwin.so: mach-o, but wrong architecture

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=100, max=1000), stop=stop_after_attempt(6))
def completion_with_backoff(prompt):
    #time.sleep(1)
    return get_completion(prompt, model="gpt-3.5-turbo")


In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    #print(response.choices[0].message["content"])
    return response.choices[0].message["content"]

def formatted_to_dataframe(string):
  lines = string.split("\n")
  data = []

  for line in lines:
    #print(line)
    #if len(line) >= 10:
    if len(line) >= 18:
      parts = line.split("/")
      print(len(line))
      print(line)
      aspect = parts[0].strip()
      polarity = parts[1].strip()
      summary = parts[2].strip()
      data.append([aspect, polarity, summary])

  df = pd.DataFrame(data, columns=["Aspect", "Polarity", "Summary"])
  return df


In [ ]:
review = """ Had trouble connecting initially. I called customer service.
They rebooted the machine and it worked just fine after that. It is a paid service.
Close to the Papa Murphys Pizza store. Uncovered. Would be accessible at any time day or night.
Well lit parking lot. This is one of the better charging stations.
Seems to be cheaper sometimes n less crowded, maybe? They’re all pretty crowded.
But there’s a Target right there, walking distance, so you can shop n charge if you want.
This is in a residential apt complex with a toddler day care in the middle of it all, just FYI.
There are also other chargers with different plug-ins, I’m not sure what kind, I haven’t checked it out. """
def get_response(review):
  prompt = f"""
  Classify the review and recognize all aspects terms with their corresponding opinion terms and sentiment polarity in [aspect/polarity/summary] format.
  Use the aspects provided: [Accessibility and Accessibility, Amenities and location, Compatibility and Connectivity, Charging speed and efficiency, reliability and maintenance, price and cost, queue and waiting time, customer service, ease of use, payment option, Safety, user interface, mobile app].
  Now I will give you the following examples so that you will be able to better classify the reviews

Review 1: [16 Tesla superchargers and 4 other ev chargers located in the same parking lot as target, ronin, scissors & scotch, yoga six, blue sky, Logan house coffee, crisp & green, mod pizza, I scream gelato, Torchys and chase bank!]
 Amenities and Location/Positive/target, ronin, scissors & scotch, yoga six, blue sky, Logan house coffee, crisp & green, mod pizza, I scream gelato, Torchys and chase bank!]

Review 2:  [16 Tesla superchargers and 4 other ev chargers located in the same parking lot as target, ronin, scissors & scotch, yoga six, blue sky, Logan house coffee, crisp & green, mod pizza, I scream gelato, Torchys and chase bank!]
 Availability and Accessibility/positive/16 Tesla superchargers and 4 other ev chargers

Review 3: [The Tesla Supercharger was fast, convenient, and easy to use! Charging my car was a breeze, and the location was great. Highly recommend for Tesla owners.
 ease of use/positive/convenient, and easy to use

Review 4: ["The Tesla Supercharger was fast, convenient, and easy to use! Charging my car was a breeze, and the location was great. Highly recommend for Tesla owners.]
 charging speed and efficiency/Positive/fast and convenient

Review 5: [This is the worst EA charger. It constantly has one or two chargers (out of 3) not working and there are long lines of cars waiting to charge.]
 reliability and maintenance/Negative/It constantly has one or two chargers (out of 3) not working

Review 6: [This charging station is fairly conveniently located. It's within a parking area for Bank of America, but it's not limited to customer parking. This might be a spot where if you wanted to charge and grab a quick bite nearby, you could, but I generally haven't done that myself. This charging station is generally reliable, although I have had issues a few times where I've had to contact customer support to get going — although to be fair, that's not necessarily an issue isolated to this charging station.]
 customer service/Positive/although I have had issues a few times where I've had to contact customer support to get going

Review 7: [My very first public charging experience - don't think it could have been any better. I hooked up, it identified my car, knew I had a credit through Ford and away it went. Added about 36%, 100 miles, 27kwh in 25 minutes. Will definitely be back.]
 Compatibility and Connectivity/Negative/I hooked up, it identified my car

Review 8: [Debit and credit cards accepted]
 payment option/Positive/Debit and credit cards accepted

Review 9: [$0.34 per kwhr as of 9/22/2022, which is one of cheapest rates on the I-95 corridor currently. Only downside is that Cracker Barrel is only food in walking distance, if only want a snack. But plenty empty chargers so should not have idle chargers if want a full meal]
 price and cost/Positive/one of the cheapest

Review 10: [Problem is that there are lots of chargers but only one is the Chademo that I need. The Nissan Leaf also takes this connection so I had to wait for 20 minutes for it to be free. Meanwhile there were 20 or so unused chargers available.  Maybe better to have a more equal number?  Or even 2 it 3 Chademo?]
 queue and waiting time/Negative/I had to wait for 20 minutes for it to be free

Review 11: [Secure with a guard and security gate]
 safety/Positive/Secure with a guard and security gate

Review 12: [eventually I was able to charge with the touchscreen instead of the app]
 user-interface of charger and mobile app/Positive/able to charge with the touchscreen
  Review text: '''{review}'''
  """
  response = get_completion(prompt)
  return response
response = get_response(review)
print(response)

Accessibility and Accessibility/Positive/Close to the Papa Murphys Pizza store, Would be accessible at any time day or night, Well lit parking lot, This is one of the better charging stations
Amenities and location/Positive/There’s a Target right there, walking distance, so you can shop n charge if you want, This is in a residential apt complex with a toddler day care in the middle of it all
Reliability and maintenance/Positive/They’re all pretty crowded
Compatibility and Connectivity/Positive/There are also other chargers with different plug-ins
Customer service/Positive/Had trouble connecting initially. I called customer service. They rebooted the machine and it worked just fine after that
Price and cost/Positive/Seems to be cheaper sometimes n less crowded, maybe?
Queue and waiting time/Positive/They’re all pretty crowded


In [ ]:
from google.colab import files
#upload_file = files.upload()

data = pd.read_csv('/content/labeled_text.csv') #upload the file Colab

In [ ]:
reviews = data['text'].dropna()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [ ]:
reviews = reviews.reset_index(drop=True)
reviews

0       Very busy, but the only Supercharger in town. ...
1       Ok, 3a wasn't working well for me. It stopped ...
2       One charger was out when I went but the rest w...
3       Great location! It's just off the freeway behi...
4       Supercharger is in the Oxford Suites hotel par...
                              ...                        
1474    I would rate 0 stars if I could. I prefer gas ...
1475    5 stars for the 4 fast DC charging spots to ch...
1476    A nice ChargePointDCFC station with both CCS &...
1477    What can I say a quick free electric charge fr...
1478                   Thank you, great charging station!
Name: text, Length: 1479, dtype: object

In [ ]:
#pd.options.display.max_colwidth = 300

In [ ]:
# reviews2 = reviews[5:10]
# rewiews3  = '. '.join(reviews2)
# rewiews3 = f"\'''{rewiews3}\'''"

In [ ]:
results = pd.DataFrame(columns=['Aspect', 'Polarity', 'Summary'])
i = 0
for item in tqdm(reviews):
  try:

    response = get_response(item)

    if ('No feedback' not in response) and ('no feedback' not in response):
      # res = response.split(',')
      # print(type(response))
      #res = response.strip('][').split('\n')
      # print(res)
      df = formatted_to_dataframe(response)

      df['Review'] = item
      results = pd.concat([results, df], ignore_index=True)
      i = i + 1
      if i%1000 == 0:
        results.to_csv('results_ABSA.csv', index=False)
  except Exception as e:
        # Handle the specific exception that occurred
      print(f"Error occurred: {e}")
results

  0%|          | 1/1479 [00:04<1:54:28,  4.65s/it]

83
Accessibility and Availability/Neutral/Very busy, but the only Supercharger in town
84
Amenities and Location/Neutral/10 minutes to Starbucks so easy walk while you charge
141
Reliability and Maintenance/Neutral/The app reported it was closed, but there are only a couple being serviced by Tesla while we are charging
40
Queue and Waiting Time/Neutral/Very busy
130
Customer Service/Neutral/The app reported it was closed, but there are only a couple being serviced by Tesla while we are charging
73
Ease of Use/Neutral/10 minutes to Starbucks so easy walk while you charge
124
Mobile App/Neutral/The app reported it was closed, but there are only a couple being serviced by Tesla while we are charging


  0%|          | 2/1479 [00:07<1:22:36,  3.36s/it]

104
Charging speed and efficiency/Negative/3a wasn't working well for me. It stopped charging several times.
102
Reliability and maintenance/Negative/3a wasn't working well for me. It stopped charging several times.
43
Queue and waiting time/Neutral/Pretty busy.
64
Amenities and location/Neutral/Bathroom is in basement of hotel.


  0%|          | 3/1479 [00:08<1:01:20,  2.49s/it]

...

100%|█████████▉| 1478/1479 [52:52<00:01,  1.21s/it]

102
Charging speed and efficiency/Positive/quick free electric charge from 20% to 80% in about a half hour
51
Customer service/Positive/people they were friendly
59
Ease of use/Positive/able to get in and out with no problem


100%|██████████| 1479/1479 [52:56<00:00,  2.15s/it]

27
Overall sentiment: Positive
Error occurred: list index out of range


,Aspect,Polarity,Summary,Review
0,Accessibility and Availability,Neutral,"Very busy, but the only Supercharger in town","Very busy, but the only Supercharger in town. ..."
1,Amenities and Location,Neutral,10 minutes to Starbucks so easy walk while you...,"Very busy, but the only Supercharger in town. ..."
2,Reliability and Maintenance,Neutral,"The app reported it was closed, but there are ...","Very busy, but the only Supercharger in town. ..."
3,Queue and Waiting Time,Neutral,Very busy,"Very busy, but the only Supercharger in town. ..."
4,Customer Service,Neutral,"The app reported it was closed, but there are ...","Very busy, but the only Supercharger in town. ..."
...,...,...,...,...
2198,Accessibility and Availability,Negative,awful the pedestrian Accessibility+Availability,5 stars for the 4 fast DC charging spots to ch...
2199,Accessibility and Amenities and Location,Positive,"Blue Ridge Energies parking lot, Blue Ride Ene...",A nice ChargePointDCFC station with both CCS &...
2200,Charging speed and efficiency,Positive,quick free electric charge from 20% to 80% in ...,What can I say a quick free electric charge fr...
2201,Customer service,Positive,people they were friendly,What can I say a quick free electric charge fr...


# New Section

In [ ]:
results.to_csv('/content/pred_results.csv', index=False)

In [ ]:
results.to_csv('/content/drive/MyDrive/results_ABSA.csv', index=False)